# Generating Headlines in English Using LSTM

In today’s fast-paced digital world, the ability to create compelling and relevant headlines is crucial for capturing audience attention and driving engagement. Headlines serve as the first impression of content, influencing readers' decisions to explore articles further. Given the growing volume of content and the demand for timely information, automating the headline creation process presents a significant opportunity.

This project addresses this need by leveraging advanced machine learning techniques to automate the generation of headlines. The core of this approach is based on Long **Short-Term Memory (LSTM) networks, a type of Recurrent Neural Network (RNN)** renowned for its ability to handle sequences and long-term dependencies in data.



## Why LSTM for Headline Generation?

Traditional algorithms for text generation often struggle with maintaining coherence over longer sequences, leading to headlines that may lack relevance or readability. LSTMs, with their specialized architecture, are designed to remember and use contextual information from earlier parts of the sequence. This makes them particularly effective for generating text that is not only grammatically correct but also contextually appropriate.

## Project Goals

The objective is to develop an **LSTM-based model** that can generate high-quality, engaging headlines in English. By training the model on a diverse dataset of existing headlines, we aim to produce headlines that are not only accurate but also creative and relevant. This model has the potential to assist content creators, journalists, and marketers by providing them with a tool to quickly generate impactful headlines, thereby enhancing productivity and content engagement.

## 1. Reading the dataset

In [3]:
with open("/content/dataset.txt", encoding="latin-1") as f:
    dataset = f.read().splitlines()

In [4]:
dataset[:10]

['New energy law promises to revolutionize the electric sector',
 'Climate change continues to be a global threat',
 'Investors seek opportunities in renewable energy',
 'Demand for electric vehicles increases',
 'COVID-19 vaccines: When will we all be protected?',
 'The debate over vaccines continues to divide opinions',
 'Health experts analyze the effectiveness of vaccines',
 'Mass vaccination against coronavirus underway',
 'Cryptocurrency market soars to new heights',
 'Is Bitcoin the currency of the future?']

## 2. Data Preparation

### 2.1 Data Cleaning

In [5]:
import string
import unicodedata

def clean_and_normalize_text(txt):
    # Remove punctuation and convert to lowercase
    txt = "".join(c for c in txt if c not in string.punctuation).lower()
    # Normalize unicode characters and encode to ASCII
    txt = unicodedata.normalize('NFKD', txt).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return txt

In [6]:
dataset = [clean_and_normalize_text(headline) for headline in dataset]

In [7]:
dataset[:10]

['new energy law promises to revolutionize the electric sector',
 'climate change continues to be a global threat',
 'investors seek opportunities in renewable energy',
 'demand for electric vehicles increases',
 'covid19 vaccines when will we all be protected',
 'the debate over vaccines continues to divide opinions',
 'health experts analyze the effectiveness of vaccines',
 'mass vaccination against coronavirus underway',
 'cryptocurrency market soars to new heights',
 'is bitcoin the currency of the future']

The function `clean_and_normalize_text` is designed to prepare text data for further processing by cleaning and standardizing it. This is a crucial step in text analysis and natural language processing. The function accomplishes the following:

**Remove Unwanted Characters:**

* **Objective:** Eliminate punctuation marks from the text.
* **Why:** Punctuation can interfere with text analysis tasks such as text classification or tokenization. Removing it **helps in focusing on the core content of the text.**

**Standardize Text:**

* **Objective:** Normalize the text by converting it to lowercase and removing any special or accented characters.
* **Why:** Converting the text to lowercase ensures uniformity, as "Hello" and "hello" would be treated as the same word. Normalizing accents and special characters helps in handling text from different sources and languages consistently, making it easier to analyze and compare.

`clean_and_normalize_text` transforms raw text into a cleaner, more uniform format. This preprocessing step is essential for effective text analysis, improving the accuracy and reliability of subsequent processing tasks such as machine learning model training or text-based querying.

### 2.2. Data Tokenization

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

In [9]:
def generate_token_sequences(tokenizer, dataset):
    # Build the tokenizer
    tokenizer.fit_on_texts(dataset)
    total_words = len(tokenizer.word_index) + 1

    # Tokenize the text in the dataset
    dataset_tokens = []
    for text in dataset:
        text_tokens = tokenizer.texts_to_sequences([text])[0]
        # Generate n-grams from the tokenized text
        for i in range(1, len(text_tokens)):
            n_gram = text_tokens[:i+1]
            dataset_tokens.append(n_gram)

    return dataset_tokens, total_words


In [10]:
dataset_tokens, total_words = generate_token_sequences(tokenizer, dataset)

The function `generate_token_sequences` is used to preprocess a text dataset by tokenizing the text and generating n-grams. It first builds a tokenizer based on the dataset, then converts each text into sequences of integers, and finally creates and collects various n-grams (sub-sequences) from the tokenized text. This preprocessing step is essential for transforming raw text data into a structured format **suitable for training machine learning models or performing further text analysis.**

The purpose of this section is to prepare the target labels for each sequence in the dataset. In the context of sequence-based models, such as those used for text generation, the labels represent the expected output for each input sequence.


**Understanding Sequences and Labels:**

* **Sequences:** The input data consists of sequences of tokens (e.g., words or characters) where each sequence is used to predict the next token.
* **Labels:** **The label for each sequence is the token that immediately follows the sequence**. For example, if the sequence is "The cat is", the label would be "on" if the full text was "The cat is on the mat".

**Generating Input Sequences and Labels:**

* **Input Sequences:** Each sequence of tokens is used to predict the next token. To prepare this, each sequence needs to be split so that the model can learn from the sequence to predict the next token.
* **Labels:** For each sequence, the corresponding label is the next token in the sequence. The model will learn to map the input sequence to this label.
Implementation Details:

**Here’s a step-by-step breakdown of how this is generally implemented:**

**Determine Maximum Sequence Length:**

Calculate the length of the longest sequence in the dataset to ensure that all sequences are padded to the same length.
* **Apply Padding:**
Use padding to standardize the length of all sequences. Padding involves adding zeros (or another value) to the beginning or end of sequences to make them all the same length.
* **Split Sequences:**

Separate each padded sequence into two parts:
* **Input Part:** All tokens except the last one.
* **Label Part:** The last token in the sequence.

**Convert Labels to One-Hot Encoding:**

If the task is classification, convert the labels into a one-hot encoded format where each label is represented as a binary vector indicating the presence of a token in the vocabulary.

In [11]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.utils as ku

def prepare_sequences_and_labels(dataset_tokens, total_words):
    # Determine the length of the longest sequence in the dataset
    max_sequence_len = max([len(text) for text in dataset_tokens])

    # Apply padding to all sequences to ensure they are of the same length
    dataset_tokens = np.array(pad_sequences(dataset_tokens, maxlen=max_sequence_len, padding='pre'))

    # Generate input features and target labels
    X_train, y_train = dataset_tokens[:, :-1], dataset_tokens[:, -1]

    # One-hot encode the labels
    y_train = ku.to_categorical(y_train, num_classes=total_words)

    return X_train, y_train, max_sequence_len


In [12]:
X_train, y_train, max_sequence_len = prepare_sequences_and_labels(dataset_tokens, total_words)

In [13]:
max_sequence_len

50

The function `prepare_sequences_and_labels` is used to preprocess token sequences for model training. It pads sequences to ensure uniform length, generates input features and target labels from the sequences, and converts labels into a one-hot encoded format. This preprocessing step is essential for preparing data in a format suitable for training machine learning models, particularly in tasks such as sequence prediction.

3. Model Architecture and Design for Sequence Prediction

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, BatchNormalization

In [15]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1

    model = Sequential()

    # Add Input Embedding Layer
    model.add(Embedding(input_dim=total_words, output_dim=50, input_length=input_len))

    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.2))

    # Add Hidden Layer 2 - LSTM Layer
    model.add(LSTM(64))
    model.add(Dropout(0.2))

    # Add Batch Normalization
    model.add(BatchNormalization())

    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

model = create_model(max_sequence_len, total_words)
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

The model we’ve designed is aimed at handling sequence prediction tasks, like generating text. It utilizes several advanced techniques to make the most out of the sequence data.

Starting with the **Embedding Layer**, we convert our tokens (words or characters) into **dense vector representations**. This layer maps each unique token in our vocabulary to a high-dimensional vector space. We’ve chosen an embedding dimension of 50, which means each token is represented by a 50-dimensional vector. This allows the model to capture and learn subtle semantic meanings and relationships between different tokens.

Next, **we have two LSTM (Long Short-Term Memory) layers**. LSTMs are a type of recurrent neural network designed to handle sequences of data, making them ideal for tasks where understanding context over time is crucial, such as predicting the next word in a sentence.

The first LSTM layer has 128 units and is set to return sequences. This means that instead of outputting just the final state of the sequence, it outputs the state at each time step. This is important because we want to pass the sequence of outputs to the next LSTM layer. To prevent overfitting, which can occur when the model learns the training data too well but performs poorly on unseen data, we apply a dropout rate of 20%. Dropout randomly disables a fraction of neurons during training, which forces the model to learn more robust features.

Following the first LSTM layer, we add a second LSTM layer with 64 units. This layer processes the sequences output by the first LSTM layer and helps in refining the learned patterns. **Again, we apply a 20% dropout rate to ensure the model generalizes well.**

To further stabilize and enhance training, we include a Batch Normalization layer. Batch normalization normalizes the output of the previous layer by adjusting and scaling the activations. This technique helps in accelerating training and achieving better model performance by reducing internal covariate shift.

Finally, we arrive at the **Dense Output Layer**. This layer has as many units as there are unique tokens in our vocabulary, each with a softmax activation function. The softmax function converts the raw output into probabilities, indicating how likely each token is to follow the given sequence. **This output layer enables the model to make predictions about what the next token should be based on the learned sequences.**

When we compile the model, we use c**ategorical crossentropy as the loss function, which is suitable for multi-class classification problems like this**. **The Adam optimizer is chosen for its efficiency and ability to handle sparse gradients**. We also track accuracy as a metric to monitor how well the model is performing during training.

Overall, this architecture is designed to capture the temporal dependencies in sequences through LSTM layers, while techniques like dropout and batch normalization enhance its ability to generalize and perform effectively on unseen data. This setup aims to be robust and effective for generating meaningful sequences or text predictions.

In [18]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 28s 99ms/step - accuracy: 0.4116 - loss: 2.8751
Epoch 2/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 28s 100ms/step - accuracy: 0.4498 - loss: 2.6033
Epoch 3/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 43s 108ms/step - accuracy: 0.4729 - loss: 2.4324
Epoch 4/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 29s 100ms/step - accuracy: 0.5063 - loss: 2.2352
Epoch 5/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 28s 99ms/step - accuracy: 0.5365 - loss: 2.0552
Epoch 6/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 30s 104ms/step - accuracy: 0.5651 - loss: 1.8905
Epoch 7/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 42s 107ms/step - accuracy: 0.5755 - loss: 1.8134
Epoch 8/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 29s 100ms/step - accuracy: 0.6036 - loss: 1.6762
Epoch 9/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 41s 99ms/step - accuracy: 0.6080 - loss: 1.5897
Epoch 10/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 41s 99ms/step - accuracy: 0.6171 - loss: 1.5378
Epoch 11/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 42s 103ms/step - accuracy: 0.6457 - loss: 1.4468
Epoch 12/100

## Model Evaluation and Prediction

In [19]:
def generate_text_from_prompt(prompt, num_words, model, tokenizer, max_sequence_len):
    generated_text = prompt

    for _ in range(num_words):
        # Preprocess the prompt
        prompt_proc = clean_and_normalize_text(generated_text)
        prompt_proc = tokenizer.texts_to_sequences([prompt_proc])[0]
        prompt_proc = pad_sequences([prompt_proc], maxlen=max_sequence_len-1, padding='pre')

        # Predict the next word
        predict = model.predict(prompt_proc, verbose=0)
        predicted_index = np.argmax(predict, axis=1)[0]

        # Convert predicted index to word
        next_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                next_word = word
                break

        # Append the predicted word to the generated text
        generated_text += " " + next_word

    return generated_text.title()

The `generate_text_from_prompt` function is designed to generate text based on an initial prompt provided by the user. Here's how it works:

* **Starting Point:** The function begins with a **user-provided prompt**, which serves as the seed for generating new text. This prompt is passed to the function along with the number of words to generate, the trained model, the tokenizer used during training, and the maximum sequence length the model expects.

* **Text Preparation:** Before the model can make predictions, the prompt needs to be preprocessed. This involves cleaning the text by removing punctuation and converting it to lowercase. The preprocessed text is then tokenized using the same tokenizer that was used to train the model. This process transforms the text into sequences of integers that represent words.

* **Generating Words:** With the processed prompt, the function then prepares the input for the model. It does this by padding the sequence to ensure it matches the length expected by the model. Padding is crucial because the model requires a fixed input size, and padding helps to standardize the length of sequences.

* **Making Predictions:** The model takes this padded sequence and predicts the next word in the sequence. The output is a probability distribution over the vocabulary, indicating the likelihood of each word being the next one. The function identifies the word with the highest probability as the predicted next word.

* **Mapping Index to Word:** After predicting the next word, the function needs to convert the predicted index back into the actual word. It does this by looking up the word index dictionary provided by the tokenizer, which maps integer indices to their corresponding words.

* **Appending the Word:** The newly predicted word is then appended to the current text, extending the prompt with the generated word. This updated text now becomes the new prompt for generating subsequent words.

* **Repeating the Process:** The function repeats this process of predicting the next word and appending it to the text for the number of words specified by the user. This iterative process gradually builds up the text based on the initial prompt.

* **Formatting the Output:** Once the desired number of words has been generated, the function formats the final text by capitalizing the first letter of each word, providing a cleaner and more readable output.

In essence, this function is like a creative writing assistant that takes a starting sentence and continues to build upon it word by word. By leveraging the trained model's ability to predict the next word in a sequence, it generates coherent and contextually relevant text based on the initial prompt. This is particularly useful in applications like text completion, story generation, or any scenario where automated text creation is required.

In [27]:
print(generate_text_from_prompt("The Cybersecurity", 8, model, tokenizer, max_sequence_len))

The Cybersecurity Of The Future In Limited Spaces Enhancing Quality


In [34]:
print(generate_text_from_prompt("Artificial Intelligence", 3, model, tokenizer, max_sequence_len))

Artificial Intelligence In Education Personalizing


In [37]:
print(generate_text_from_prompt("The future", 7, model, tokenizer, max_sequence_len))

The Future Of Education Driven By Ai And Chains


In [38]:
print(generate_text_from_prompt("Blockchain", 8, model, tokenizer, max_sequence_len))

Blockchain Technology And Its Impact On The Financial Industry


In [41]:
print(generate_text_from_prompt("Ethical Hacking", 8, model, tokenizer, max_sequence_len))

Ethical Hacking Fashion A Trend That Changes The Game Industry


## Saving the Trained Model and Weights

After training a deep learning model, it's essential to save both the model architecture and its trained weights for future use or deployment. We can achieve this in TensorFlow using the save method provided by the Keras API.

In [29]:
# Define the file paths for saving the model and weights
model_path = 'trained_model.model.h5'
weights_path = 'trained_model.weights.h5'

# Save the trained model architecture
model.save(model_path)

# Save the trained model weights
model.save_weights(weights_path)

This code saves the entire model architecture to a single HDF5 file (model.h5) and the trained weights to another HDF5 file (weights.h5). These files can then be loaded later to make predictions on new data or continue training the model.